In [67]:
import pandas as pd
df = pd.read_csv('Riskv13.csv')

In [72]:
pd.set_option('display.max_columns', None)  # show all columns
pd.set_option('display.width', 200)         # adjust based on terminal width
pd.set_option('display.float_format', '{:.3f}'.format)  # float precision

In [52]:
team = pd.read_csv('Alleactionsv2.csv')


In [54]:
df = df.merge(team[['original_event_id', 'pass_length']], on='original_event_id', how='left')

In [55]:
team = team.drop_duplicates(subset='player_id')
df = df.merge(team[['player_id', 'team_name']], on='player_id', how='left')

In [57]:
result = df[['original_event_id', 'player_name','team_name', 'starting_position', 'pass_length', 'result_id', 'predicted_prob', 'predicted_class']]


In [ ]:
def first_non_substitute(positions):
    for position in positions:
        if position != 'Substitute':
            return position
    return 'Substitute'  # Default if all are 'Substitute'

average_predicted_prob = result.groupby('player_name').agg(
    team=('team_name', 'first'),
    average_predicted_prob=('predicted_prob', 'mean'),
    number_of_passes=('predicted_prob', 'size'),
    starting_position=('starting_position', lambda x: first_non_substitute(x)),
    average_pass_distance=('pass_length', 'mean'),
).reset_index()

# Filter players with more than 50 passes
average_predicted_prob = average_predicted_prob[average_predicted_prob['number_of_passes'] > 50]

# Sort by average_predicted_prob
average_predicted_prob = average_predicted_prob.sort_values(by='average_predicted_prob', ascending=True)

print(average_predicted_prob)

In [61]:
average_predicted_prob.to_csv('average_predicted_prob.csv', index=False, encoding='utf-8-sig')

In [ ]:
average_predicted_prob = result.groupby('team_name').agg(
    average_predicted_prob=('predicted_prob', 'mean'),
    number_of_passes=('predicted_prob', 'size'),
    average_length_pass=('pass_length', 'mean'),
).reset_index()

# Filter players with more than 50 passes
average_predicted_prob = average_predicted_prob[average_predicted_prob['number_of_passes'] > 50]

# Sort by average_predicted_prob
average_predicted_prob = average_predicted_prob.sort_values(by='average_predicted_prob', ascending=True)

print(average_predicted_prob)
average_predicted_prob.to_csv('average_predicted_prob_team.csv', index=False, encoding='utf-8-sig')

In [45]:
overall_average_predicted_prob = result['predicted_prob'].mean()
print(f"Overall Average Predicted Probability: {overall_average_predicted_prob:.6f}")

Overall Average Predicted Probability: 0.836328


In [74]:
percentage_result_id_1 = (df['result_id'].value_counts(normalize=True)[1] * 100)
print(f"Percentage of times result_id = 1: {percentage_result_id_1:.2f}%")

Percentage of times result_id = 1: 83.49%
